In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from pytz import timezone
import pytz
from netCDF4 import Dataset, num2date
from dateutil.rrule import *
import os
import netCDF4 as nc
import calendar as cal

In [15]:
data1 = Dataset('data_wind_10_11_125.nc','r')
# data = Dataset('data_wind_Ene_Mar_2020.nc','r')

In [16]:
%%time
lat = np.array(data1.variables['latitude'][:])
lon = np.array(data1.variables['longitude'][:])
nctime = np.array(data1.variables['time'][:])
t_unit = data1.variables['time'].units
t_cal = data1.variables['time'].calendar
time = num2date(nctime,units=t_unit,calendar=t_cal)#Date time
data_u1 = data1.variables['u10'][:]
data_v1 = data1.variables['v10'][:]

CPU times: total: 0 ns
Wall time: 14.4 ms


In [17]:
# time = np.asarray(rrule(HOURLY, dtstart=dt.datetime(2020,1,1,0), until=dt.datetime(2020,3,31,23), interval=1)[:])
# data_u = data.variables['u10'][:]
# data_v = data.variables['v10'][:]

In [18]:
# # ***********************************************************
# # Cargar datos U y V
# # ***********************************************************
# time = np.asarray(rrule(HOURLY, dtstart=dt.datetime(2019,10,1,0), until=dt.datetime(2019,11,30,23), interval=1)[:])
# data = nc.Dataset(r'data_wind_detalle_Nov_0_025.nc')
# data_u = data.variables['u10'][:]
# data_v = data.variables['v10'][:]
# ndays = cal.monthrange(2019, 11)[1]
# ndays

In [19]:
utc = pytz.utc
col_zone = timezone('America/Bogota')
time2=np.array([datetime(i.year,i.month,i.day,i.hour,i.minute,tzinfo=utc) for i in time])
loc_co =np.array([i.astimezone(col_zone) for i in time2])
time1 = np.array([datetime(i.year,i.month,i.day,i.hour,i.minute) for i in loc_co])
time1

array([datetime.datetime(2021, 9, 30, 19, 0),
       datetime.datetime(2021, 9, 30, 20, 0),
       datetime.datetime(2021, 9, 30, 21, 0), ...,
       datetime.datetime(2021, 11, 30, 16, 0),
       datetime.datetime(2021, 11, 30, 17, 0),
       datetime.datetime(2021, 11, 30, 18, 0)], dtype=object)

In [22]:
fecha_inicio_19 = datetime(2021,10,28,0,0)
fecha_fin_19 = datetime(2021,11,14,16,0)
in_finicio_19= np.where(time1==fecha_inicio_19)[0][0]
in_ffin_19 = np.where(time1==fecha_fin_19)[0][0]
data_u_split = data_u1[in_finicio_19:in_ffin_19+1,:,:]
data_v_split = data_v1[in_finicio_19:in_ffin_19+1,:,:]
time_split = time1[in_finicio_19:in_ffin_19+1]

In [23]:
#Se hace el recorte del vector
q1=np.where(np.asarray(time_split[:])==time_split[0])[0]
q2=np.where(np.asarray(time_split[:])==time_split[-1])[0]
with open ("wind_november_4_14_2021.wnd",'w') as f:# Aquí se crea el archivo.txt
    for i in range(q1[0],q2[0]):
        f.write("%s\n" % time_split[i].strftime("%Y%m%d %H%M%S"))
        np.savetxt(f,data_u_split[i,:,:])#,fmt='%10.4f')
        np.savetxt(f,data_v_split[i,:,:])#,fmt='%10.4f')